In [ ]:
import numpy as np
from numpy.linalg import norm
from numpy import dot

In [ ]:
def main():
    print('Recommendation System')
    print('Please select operation\n\
    1. Item based\n\
    2. User based\n\
    3. Hybrid')
    k=int(input('Choice:'))
    n = int(input('Please enter the number of clusters:'))
    id=int(input('Enter user id:'))
    if k == 1:
        print('Please wait\nGenerating Recommendations......')
        arr=item_based(id,5,n)
        printing(arr)
    elif k==2:
        print('Please wait\nGenerating Recommendations......')
        arr=user_based(id,5,n)
        printing(arr)
    elif k==3:
        print('Please wait\nGenerating Recommendations......')
        arr=hybrid(id,n)
        printing(arr)
    else:
        print('wrong input , try again!')
        return main()
    return 

def hybrid(id,n):
    arr=[]
    print('Collaborative Filtering: Hybrid mode')
    arr1 = user_based(id,5,n)
    arr.extend(arr1)
    arr2= item_based(id,5,n)
    arr.extend(arr2)
    arr.sort(key=lambda x : x[1],reverse=True)
    return arr[:20]

def printing(arr):
    print('TOP 20 MOST RECOMMENDED MOVIES FOR CHOSEN USER')
    print('Movie title\t\t\tRating')
    for a in arr:
        print(a)
        

In [ ]:
def item_based(id,n_hood,k):
    movie_dic=[]
    for i in range(k):
        dfn=[]
        with open('cluster'+str(i)+'.csv','r',buffering=4000000,encoding='utf-8') as file:
                for line in file:
                    dfn.append(line.replace('\n','').split(','))
        movie_dic.extend(item_wise(id,n_hood,dfn))
        #print(i)
    return sorted(movie_dic,key=lambda x: x[1],reverse=True)[:20]   #top 20 highly rated movies

In [ ]:
def item_wise(id,n_hood,dfn):
    rated=[]
    unrated=[]
    rates=[]
    #id=9
    for q in dfn:
        if q[id]!='':
            rated.append(q)
            rates.append(float(q[id]))
        else:
            unrated.append(q)
    #print(len(rated))
    movies = {}
    for y in unrated:
        sim=[]
        if len(rated)!=0:
            if n_hood>len(rated):
                n_hood=len(rated)      #making sure neighboorhood is not too large
            for x,r in zip(rated,rates):
                sim.append([pearson_corr(x[1:],y[1:])])  # sim, where is rate?
                sim[-1].append(r)
            sim_sorted=sorted(sim,key=lambda x:x[0],reverse=True)
            rating = weighted_mean2(sim_sorted[:n_hood])
            movies.update({y[0]:rating})
        else:
            movies.update({y[0]:1.0000})
    return sorted(movies.items(),key=lambda x : x[1],reverse=True)[:20] #returning top 20 movies for cluster is list format

In [ ]:
def user_wise(id,n_hood,dfn,user_vecs):
    rated_id=[]
    unrated=[]
    for i in range(len(dfn)):
        if dfn[i][id]=='':
            unrated.append(dfn[i])     #movies not rated by user
            rated_id.append(i)
    movies={}
    y=user_vecs[id-1]        #user vector

    for i in range(len(unrated)):
        #print(i)
        rated=[user for user in user_vecs if user[rated_id[i]]!='']  # uv x if uv x has smtx in pot i
        rates=[float(r[rated_id[i]]) for r in rated]
        sim=[]
        if len(rated)!=0:
            if n_hood>len(rated):
                n_hood=len(rated)      #making sure neighboorhood is not too large
            for x,r in zip(rated,rates):
                sim.append([pearson_corr(x,y)])  # sim, with user vectors
                sim[-1].append(r)
            sim_sorted=sorted(sim,key=lambda x:x[0],reverse=True)
            rating = weighted_mean2(sim_sorted[:n_hood])
            movies.update({unrated[i][0]:rating})
        else:
            movies.update({unrated[i][0]:1.0000})
    return sorted(movies.items(),key=lambda x : x[1],reverse=True)[:20] #returning top 20 movies for cluster is list format

In [ ]:
def pearson_corr(x,y):
    xval = meanval(x)
    yval = meanval(y)
    #print(sum(yval))
    return cosine_similarity(xval,yval)


def cosine_similarity(v1,v2):
    d=(norm(v1)*norm(v2))
    #if d==0:
    #return 0 #to handle the case of a very small dinominator
    return dot(v1, v2)/d

In [ ]:
import numpy as np
def meanval(y):
    re=np.zeros(len(y))
    x=np.array(y)
    id_val=np.where(x!='')
    xr=np.array(x[id_val],dtype='float64')
    if len(xr)!=0:
        m=sum(xr)/len(xr)
        xr-=m
    x[np.where(x=='')]=0
    re[id_val]=xr
    return re

In [ ]:
def weighted_mean(sims):
    summ=0
    simi = [x[0] for x in sims]
    rate = [x[1] for x in sims]
    for i in range(len(sims)):
        summ+=simi[i]*rate[i]
    v = summ/sum(simi)
    if np.isnan(v):
        return 0
    return round(v,4)

In [ ]:
def weighted_mean2(sims):
    summ=0
    simi = [x[0] for x in sims]
    rate = [x[1] for x in sims]
    for val in simi:
        if val<0:                       #a check for the existence of a false positive in the cluster
            #print('here')
            return (round(sum(rate)/len(rate),4) - 1)  #penalty
    for i in range(len(sims)):
        summ+=simi[i]*rate[i]
    v = summ/sum(simi)
    if np.isnan(v):
        return 0
    return round(v,4)

In [ ]:
def user_based(id,n_hood,k):
    movie_dic=[]
    for n in range(k):
        dfn=[]
        user_vecs=[[] for i in range(7045)]
        with open('cluster'+str(n)+'.csv','r',buffering=4000000,encoding='utf-8') as file:
                for line in file:
                    dfn.append(line.replace('\n','').split(','))
                    for i in range(7045):
                        user_vecs[i].append(dfn[-1][i+1])      #because movie title is at index 0
        movie_dic.extend(user_wise(id,n_hood,dfn,user_vecs))
        #print(n)
    return sorted(movie_dic,key=lambda x: x[1],reverse=True)[:20]

In [ ]:
main()